In [1]:
#Day 18
#
#https://adventofcode.com/2021/day/18

####################
#
#   Algorithm
#
####################

from json import *
from math import *

#compute the sum of any structure
def sum(x):
    if isinstance(x,list):
        a,b=x
        return sum(a)*3+sum(b)*2
    return x

#Add two structures
def add(a,b):
    return [a,b]

#shrapnel from explosion is sent up in tree and deposited 
def shrap(x, val, send_left):
    if isinstance(x,list):
        a,b=x
        if send_left:
            return [shrap(a, val, send_left),b]
        else:
            return [a, shrap(b, val, send_left)]
    
    #if send_left:
    #    print("shrapnel going left", val)
    #else:
    #    print("shrapnel going right", val)
    return x + val


#reduce returns: value[int], done[True|False], shrapnel_left[None|int], Shrapnel_right[None|int]
def _reduce(x, debug, allow_explode, depth=0):
    #only split single values
    if isinstance(x,int):
        if x >= 10 and not allow_explode:
            h = x/2
            hl = int(floor(h))
            hr = int(ceil(h))
            assert(hl+hr == x)
            if debug:
                print("splitting ", x, "into", [hl, hr])
            x = [hl, hr]
            return x, True, None, None
    if isinstance(x,list):
        l,r=x        
        if isinstance(l,int) and isinstance(r, int) and depth >= 4 and allow_explode: 
            #Explode!!!
            if debug:
                print("exploding", x)
            return 0, True, l, r    
        else:
            l, haschanged, sl, sr = _reduce(l, debug, allow_explode, depth+1)
            if haschanged:
                if sr:
                    r = shrap(r, sr, True)
                    sr = None
                return [l, r], haschanged, sl, sr
            
            r, haschanged, sl, sr = _reduce(r, debug, allow_explode, depth+1)
            if haschanged:
                if sl:
                    l = shrap(l, sl, False)
                    sl = None
                return [l, r], haschanged, sl, sr
            
    return x, False, None, None
    
def reduce(x, debug=False):
    running = 1
    while running:
        #do explosion. 
        #If exploded explode again
        #explosions ended do one aplit
        #go to step one
        running -=1
        x, didchange_explode, sl, sr = _reduce(x, debug, True )        
        if didchange_explode:
            running = 1
        else:
            x, did_split,   sl, sr = _reduce(x, debug, False)        
            if did_split:
                running = 1
        
    return x



####################
#
#   Test cases
#
####################

#explosion_tests
homeworks = []
homeworks.append(("[[[[[9,8],1],2],3],4]", [[[[0,9],2],3],4], None)) 
homeworks.append(("[7,[6,[5,[4,[3,2]]]]]", [7,[6,[5,[7,0]]]], None))
homeworks.append(("[[6,[5,[4,[3,2]]]],1]", [[6,[5,[7,0]]],3], None))
homeworks.append(("[[6,[5,[4,[3,2]]]],1]", [[6,[5,[7,0]]],3], None))
homeworks.append(("[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]", [[3,[2,[8,0]]],[9,[5,[7,0]]]], None)) #[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
homeworks.append(("[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]", [[3,[2,[8,0]]],[9,[5,[7,0]]]], None))

homeworks.append(("[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]", [[3,[2,[8,0]]],[9,[5,[7,0]]]], None))

#split tests
homeworks.append(("[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]", [[[[0,7],4],[[7,8],[6,0]]],[8,1]],None))

#sum tests
homeworks.append(("[[1,2],[[3,4],5]]", None, 143))
homeworks.append(("[[[[0,7],4],[[7,8],[6,0]]],[8,1]]", None, 1384))
homeworks.append(("[[[[1,1],[2,2]],[3,3]],[4,4]]", None, 445))
homeworks.append(("[[[[3,0],[5,3]],[4,4]],[5,5]]", None, 791))
homeworks.append(("[[[[5,0],[7,4]],[5,5]],[6,6]]", None, 1137))
#homeworks.append(("[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]", None, 3488))


homeworks.append(("""[1,1]
[2,2]
[3,3]
[4,4]""", [[[[1,1],[2,2]],[3,3]],[4,4]], None))

homeworks.append(("""[1,1]
[2,2]
[3,3]
[4,4]
[5,5]""", [[[[3,0],[5,3]],[4,4]],[5,5]], None))


homeworks.append(("""[1,1]
[2,2]
[3,3]
[4,4]
[5,5]
[6,6]""", [[[[5,0],[7,4]],[5,5]],[6,6]], None)) 

homeworks.append(("""[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]""", [[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]], None))
                 
                
homeworks.append(("""[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]""", [[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]], 4140))

homeworks.append((open("i18.txt").read(), None, 3486))

####################
#
#   data loading
#
####################
for i in range(len(homeworks)):
    hw, reduceto, sumto = homeworks[i]
    #homework comes in as text, one line per expression
    #expressions must be added up
    tmp = None
    for line in [loads(x) for x in hw.split("\n")]:
        if tmp is None:
            tmp  = line
        else:
            tmp = add(tmp, line)
            tmp = reduce(tmp)
          
    print(i, "#################")
    print(tmp)
    print("-----------------")
    
    #reduce expression and find sum    
    reduced=reduce(tmp)
    s = sum(reduced)
    
    all_is_fine = True
    
    
    l = ""
    if reduceto:
        l += "Should reduce to  "
        l += str(reduceto)
        if reduced == reduceto:
            l += "\nwhich it did. :)"    
        else:
            l += "\nbut it reduced to "
            l += str(reduced)
            all_is_fine = False
        print(l)
    else:
        print("reduced to:", reduced)

    if sumto:
        l = "\nShould have sum "
        l += str(sumto)
        if sumto == s:
            l += " which it did. :)"    
        else:
            l += " but had sum "
            l += str(s)
            all_is_fine = False
        print(l)
    else:
        print("Sum is: ", s)
    
    if not all_is_fine:
        print("Error discovered - running in debug mode")
        reduce(tmp, True)
        break


0 #################
[[[[[9, 8], 1], 2], 3], 4]
-----------------
Should reduce to  [[[[0, 9], 2], 3], 4]
which it did. :)
Sum is:  548
1 #################
[7, [6, [5, [4, [3, 2]]]]]
-----------------
Should reduce to  [7, [6, [5, [7, 0]]]]
which it did. :)
Sum is:  285
2 #################
[[6, [5, [4, [3, 2]]]], 1]
-----------------
Should reduce to  [[6, [5, [7, 0]]], 3]
which it did. :)
Sum is:  402
3 #################
[[6, [5, [4, [3, 2]]]], 1]
-----------------
Should reduce to  [[6, [5, [7, 0]]], 3]
which it did. :)
Sum is:  402
4 #################
[[3, [2, [1, [7, 3]]]], [6, [5, [4, [3, 2]]]]]
-----------------
Should reduce to  [[3, [2, [8, 0]]], [9, [5, [7, 0]]]]
which it did. :)
Sum is:  633
5 #################
[[3, [2, [8, 0]]], [9, [5, [4, [3, 2]]]]]
-----------------
Should reduce to  [[3, [2, [8, 0]]], [9, [5, [7, 0]]]]
which it did. :)
Sum is:  633
6 #################
[[3, [2, [8, 0]]], [9, [5, [4, [3, 2]]]]]
-----------------
Should reduce to  [[3, [2, [8, 0]]], [9, [5, 

In [2]:
def solve2(lst):
    
    lst = lst[0]
    lst = [loads(x) for x in lst.split("\n")]
    
    l = len(lst)
    r = 0
    for i in range(l):
        for j in range(l):
            if i != j:
                 r = max(r, sum(reduce(add(lst[i], lst[j]))))
                 r = max(r, sum(reduce(add(lst[j], lst[i]))))
    return r
    
#print(homeworks)    
print("test (3993)", solve2(homeworks[-2]))
      
print("real", solve2(homeworks[-1]))
print("fin")
     





test (3993) 3993
real 4747
fin
